In [1]:
# %% [markdown]
# # 👑 U-Net 生成器形状变换：沙盘推演
# 
# ## 🎯 本次推演目标
# 1.  **可视化追踪**：亲眼见证一个输入张量在您的`UnetGenerator`中，从输入到输出的完整形状变换旅程。
# 2.  **解密递归**：彻底理解`UnetSkipConnectionBlock`的递归嵌套是如何工作的。
# 3.  **洞悉拼接**：精确掌握`torch.cat`（跳跃连接）在每一层是如何改变通道数的。
# 
# **我们将完全基于您代码库中的`network.py`和默认参数（`num_downs=8`）进行推演。**

# %% [markdown]
# ## 🛠️ 第一步：复刻兵器库 - 从`network.py`中提取核心组件
# 
# 我们首先将`UnetGenerator`和其核心“积木块”`UnetSkipConnectionBlock`的代码复制到这里。为了追踪形状，我们将在`UnetSkipConnectionBlock`的`forward`方法中加入打印语句，作为我们的“侦察探针”。

# %%
import torch
import torch.nn as nn

# 这是 UnetSkipConnectionBlock 的“侦察兵”版本
# 我们在 forward 方法中加入了详细的打印语句
class InstrumentedUnetSkipConnectionBlock(nn.Module):
    """Defines the Unet submodule with skip connection.
    X -------------------identity----------------------
    |-- downsampling -- |submodule| -- upsampling --|
    """

    def __init__(self, outer_nc, inner_nc, input_nc=None,
                 submodule=None, outermost=False, innermost=False, norm_layer=nn.BatchNorm2d, use_dropout=False, level=0):
        super(InstrumentedUnetSkipConnectionBlock, self).__init__()
        self.outermost = outermost
        # 核心：记录当前积木块的层级，便于打印
        self.level = level
        self.indent = "    " * level
        
        if input_nc is None:
            input_nc = outer_nc
            
        downconv = nn.Conv2d(input_nc, inner_nc, kernel_size=4,
                             stride=2, padding=1, bias=False)
        downrelu = nn.LeakyReLU(0.2, True)
        downnorm = norm_layer(inner_nc)
        uprelu = nn.ReLU(True)
        upnorm = norm_layer(outer_nc)

        if outermost:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1)
            down = [downconv]
            up = [uprelu, upconv, nn.Tanh()]
            model = [
                *down,
                submodule,
                *up
            ]
        elif innermost:
            upconv = nn.ConvTranspose2d(inner_nc, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=False)
            down = [downrelu, downconv]
            up = [uprelu, upconv, upnorm]
            model = [
                *down,
                *up
            ]
        else:
            upconv = nn.ConvTranspose2d(inner_nc * 2, outer_nc,
                                        kernel_size=4, stride=2,
                                        padding=1, bias=False)
            down = [downrelu, downconv, downnorm]
            up = [uprelu, upconv, upnorm]

            if use_dropout:
                model = [*down, submodule, *up, nn.Dropout(0.5)]
            else:
                model = [*down, submodule, *up]

        self.model = nn.Sequential(*model)

    def forward(self, x):
        print(f"{self.indent}▶️ [层级 {self.level}] 输入形状: {x.shape}")
        
        # 如果不是最外层，则执行跳跃连接
        if self.outermost:
            print(f"{self.indent}  - 正在通过最外层模块...")
            output = self.model(x)
            print(f"{self.indent}◀️ [层级 {self.level}] 输出形状: {output.shape}")
            return output
        else:
            # self.model(x) 是数据走完一个U型子结构（先下后上）的结果
            sub_output = self.model(x)
            print(f"{self.indent}  - U型子结构输出形状: {sub_output.shape}")
            
            # torch.cat 是跳跃连接的核心
            print(f"{self.indent}  - 准备拼接 (Concat):")
            print(f"{self.indent}    - 编码器援军 (x)      : {x.shape}")
            print(f"{self.indent}    - 解码器主力 (sub_output): {sub_output.shape}")
            
            result = torch.cat([x, sub_output], 1)
            print(f"{self.indent}  - 拼接后形状: {result.shape} (通道数 = {x.shape[1]} + {sub_output.shape[1]})")
            print(f"{self.indent}◀️ [层级 {self.level}] 返回形状: {result.shape}")
            return result

# 这是 UnetGenerator 的修改版，它使用我们带“探针”的积木块
class InstrumentedUnetGenerator(nn.Module):
    def __init__(self, input_nc, output_nc, num_downs, ngf=64, norm_layer=nn.BatchNorm2d, use_dropout=False):
        super(InstrumentedUnetGenerator, self).__init__()

        # construct unet structure
        unet_block = InstrumentedUnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=None, norm_layer=norm_layer, innermost=True, level=num_downs-1)
        for i in range(num_downs - 5):
            unet_block = InstrumentedUnetSkipConnectionBlock(ngf * 8, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, use_dropout=use_dropout, level=num_downs-2-i)
        unet_block = InstrumentedUnetSkipConnectionBlock(ngf * 4, ngf * 8, input_nc=None, submodule=unet_block, norm_layer=norm_layer, level=3)
        unet_block = InstrumentedUnetSkipConnectionBlock(ngf * 2, ngf * 4, input_nc=None, submodule=unet_block, norm_layer=norm_layer, level=2)
        unet_block = InstrumentedUnetSkipConnectionBlock(ngf, ngf * 2, input_nc=None, submodule=unet_block, norm_layer=norm_layer, level=1)
        self.model = InstrumentedUnetSkipConnectionBlock(output_nc, ngf, input_nc=input_nc, submodule=unet_block, outermost=True, norm_layer=norm_layer, level=0)

    def forward(self, input):
        return self.model(input)

# %% [markdown]
# ## ⚔️ 第二步：沙盘推演 - 发起一次模拟进攻
# 
# 现在，我们将创建一个虚拟的输入图像，并将其送入我们装备了“探针”的U-Net生成器。请仔细观察下方打印的输出，它会清晰地展示数据是如何逐层下沉，又如何逐层上浮并融合的。

# %%
# --- 参数设定 (与您代码默认值一致) ---
INPUT_CHANNELS = 1     # 输入通道数 (灰度图)
OUTPUT_CHANNELS = 1    # 输出通道数 (灰度图)
NUM_DOWNS = 8          # 下采样总层数
NGF = 64               # 基础滤波器数量

# --- 实例化我们带“探针”的生成器 ---
print("🏗️ 正在构建U-Net生成器...\n")
generator = InstrumentedUnetGenerator(INPUT_CHANNELS, OUTPUT_CHANNELS, NUM_DOWNS, ngf=NGF)

# --- 创建一个虚拟输入张量 ---
# (批量, 通道, 高, 宽)
# 注意：由于有8次下采样，输入图像的宽高必须是 2^8 = 256 的倍数
BATCH_SIZE = 1
HEIGHT = 256
WIDTH = 256
dummy_input = torch.randn(BATCH_SIZE, INPUT_CHANNELS, HEIGHT, WIDTH)

print(f"⚔️ 模拟进攻开始！虚拟输入图像尺寸: {dummy_input.shape}\n")
print("="*60)
print("              U-NET 数据流追踪               ")
print("="*60)

# --- 发起进攻！---
output = generator(dummy_input)

print("="*60)
print(f"\n✅ 凯旋！最终输出图像尺寸: {output.shape}")

# --- 验证 ---
assert output.shape == dummy_input.shape
print("👍 验证成功：输出尺寸与输入尺寸完全一致。")

# %% [markdown]
# ## 📜 第三步：战情报告解读
# 
# 请您仔细阅读并分析上方单元格的输出。您会发现：
# 
# 1.  **层级关系**：打印输出的缩进清晰地展示了网络的递归调用关系。`[层级 0]` 是最外层，`[层级 7]` 是最内层的瓶颈。
# 2.  **编码器（下采样）**：观察从 `[层级 0]` 到 `[层级 7]` 的`输入形状`，您会看到空间尺寸（高和宽）是如何一步步减半的 (`256` -> `128` -> ... -> `1`)。
# 3.  **解码器（上采样）**：观察从 `[层级 7]` 开始的返回过程。在每一个层级，`U型子结构输出形状`的空间尺寸，都是其`输入形状`的一半。
# 4.  **跳跃连接**：在每一个层级（除最外层），观察`准备拼接 (Concat)`部分。您会看到“编码器援军”和“解码器主力”的空间尺寸是完全相同的，它们的通道数相加，形成了`拼接后形状`的总通道数。
# 
# 这份 Notebook 就是您勘察 U-Net 内部结构的动态地图。请反复运行、修改参数（如`NUM_DOWNS`）并观察其变化，直到您对这股数据洪流的走向了然于胸。

🏗️ 正在构建U-Net生成器...

⚔️ 模拟进攻开始！虚拟输入图像尺寸: torch.Size([1, 1, 256, 256])

              U-NET 数据流追踪               
▶️ [层级 0] 输入形状: torch.Size([1, 1, 256, 256])
  - 正在通过最外层模块...
    ▶️ [层级 1] 输入形状: torch.Size([1, 64, 128, 128])
        ▶️ [层级 2] 输入形状: torch.Size([1, 128, 64, 64])
            ▶️ [层级 3] 输入形状: torch.Size([1, 256, 32, 32])
                ▶️ [层级 4] 输入形状: torch.Size([1, 512, 16, 16])
                    ▶️ [层级 5] 输入形状: torch.Size([1, 512, 8, 8])
                        ▶️ [层级 6] 输入形状: torch.Size([1, 512, 4, 4])
                            ▶️ [层级 7] 输入形状: torch.Size([1, 512, 2, 2])
                              - U型子结构输出形状: torch.Size([1, 512, 2, 2])
                              - 准备拼接 (Concat):
                                - 编码器援军 (x)      : torch.Size([1, 512, 2, 2])
                                - 解码器主力 (sub_output): torch.Size([1, 512, 2, 2])
                              - 拼接后形状: torch.Size([1, 1024, 2, 2]) (通道数 = 512 + 512)
                            ◀️ [层级 7] 返回形状: 